# What is a Virtual Private Cloud?
A Virtual Private Cloud (VPC) resides inside of the AWS Cloud and is an isolated segment of the AWS cloud itself.

Upon creation, the only person to have access to the VPC is yourself.

VPCs are used to deploy resources within the VPC, including:
* different compute resources,
* storage,
* database, and
* other network infrastructure

This can be the starting place to provision and deploy solutions within the Cloud.

There is a limit of 5 VPCs per region per AWS account.

To create a VPC, it requires a name and define an IP address range, using a Classless Inter-Domain Routing (CIDR) block, that the VPC can use.

# Subnets
Subnets reside inside the VPC and allows segementation of the VPC infrastrucutre into multiple different networks.

This would be used to create better management for rescources, isolate certain resources from others, or to create high-availability and resilliency within the infrastructure.

CIDR block address is a range of IP addresses and this CIDR block range can have a subnet mask between a range of IP addresses from `/16` to `/28`.

For example, if we created the VPC using CIDR block 10.0.0.0/16 any subnets created subsequently need to reside in the CIDR block range.

Subnets could be public, accessible by users of the internet, or private, accessible by the private network.

There are two changes to make to the infrastructure to make a subnet public:
1. Internet Gateway - this is a managed by AWS and is attached to the VPC and acts as the gateway between the VPC and the outside world.
2. Route Table - associated with every subnet will be an associated route table. It is possible to have the same route table associated to multiple subnets. However you cannot associate more than one route table to a single subnet.

## Architecting multiple subnets
Architecting multiple subnets across a VPC creates high availability and resilence.

Adding additional subnets allows for resilency.

Deploying subnets in different regions means that if one availability zone goes down, the VPC can continue to run as the differernt layers can communicate with each other, just through different routing.

# Network Access Control Lists (NACLs)
Network Access Control Lists (NACLs) are virtual network-level firewalls that are associated to each and every subnet.<br>
They help control both ingress and egress traffic moving in and out of the VPC and between subnets.

Upon creation of a subnet, it will also associate a network control list. This NACL will allow traffic, both inbound and outbound, so it is not very secure.<br>
It is best practice to configure NACLs to only allow traffic that is necessary to come in and out of the subnet.

The same NACL can be applied to a number of subnets, but only a single NACL can be associated to one subnet.

# Security Groups
Security groups work act as a virtual firewall at an instance level.

For example running 3 subnets, 2 of which are running EC2 instances, whilst the last is running an RDS instance.<br>
We would like only one EC2 instance to talk to the RDS instance but not the other.<br>
Using security groups can allow us to do this.

# Network Address Translation (NAT)
A Network Address Translation gateway service allows private subnets to connect to services outside the VPC but external services cannot initiate a connection with those instances.

With AWS Shared Responsibility Model, instances that are managed by AWS but are private subnets, will need a way to be updated, patched, maintained whilst still staying private.

A NAT gateway sits within the public subnet, using an Elastic IP (EIP) address.<br>
This creates a route out to the Internet gateway, and therefore to the Internet.<br>
The route table of the private subnet needs to include a rule to provide a route to the NAT gateway.

A NAT gateway allows private instances to be able to access the internet while blocking connections initiated from the internet.

# Bastion Hosts
Bastion Hosts allows users to remote access an EC2 instance sitting in a private subnet.

Bastian Hosts are EC2 instances that sits within the public subnet.<br>
It needs to be very secure, hardened, and very robust but effectively it needs to tighten down to remove all and any vulnerabilities and loose access controls.

Once connected to the Bastion Host, a private key is needed to access the EC2 instances in the private subnet.<br>
SSH agent forwarding allows the storage of private keys for the instances within the private subnet on a local client.<br>
So when connected to the Bastion Host, users can then SSH using the private key to the EC2 instances that is stored on the client.

# Virtual Private Networks
A Virtual Private Network (VPN) is a secure way of connecting two remote networks across the internet.

To connect a VPC to a remote destination, say a data centre, we need a number of things:
* Virtual Gateway - much like an internet gateway, it enables the private subnets access out to the internet - managed by AWS
* Customer gateway - the endpoint to which the virtual gateway attaches to. It could be hardware or software.

Once the virtual private gateway is attached to the VPC and the customer gateway is installed, a tunnel can be intiated between the two endpoints.<br>
It can only be initiated from the customer gateway.<br>
The private subnet is not *connected* to the internet.

If the customer gateway installed supports the Border Gateway Protocol (BGP), then it will support dynamic routing.<br>
This will populate all the routes for the VPN connection without needing to implement static routing.

# Direct Connect
Direct Connect is a another method of connecting a remote location to the AWS environment without the use of the internet.<br>
Instead it uses private infrastructure to connect directly to the VPC.

A data centre connects directly to a AWS region via a direct connect facility owned and managed by an AWS partner.<br>
This allows connections to public sources like Amazon S3.

# VPC Peering
VPC Peering is the connectivity and communication between two VPCs.<br>
They could be in same or different regions.<br>
The peering connection is run and hosted on AWS infrastructure meaning it is highly resilent, there is no single point of failure, and there is not bottlenecked bandwidth.

Peering is a one-to-one connection only.<br>
Connecting to a third VPC requires peering between the two VPCs that are already connected.<br>
That is, if VPC-1 is connected to VPC-2, VPC-2 is connected to VPC-3, VPC-1 cannot connect to VPC-3 via VPC-2.<br>
VPC-1 will require its own peering connection to VPC-3.

Relating to IP addressing and peering, VPCs with identical addresses would not connect because of IP overlap.<br>
That is, if VPC-1 and VPC-3 have IP addresses of 10.0.0.0/16, they would not connect with one another.

To start a peering connection, VPC-1 (the requester), sends a VPC peering request to VPC-2 (the accepter).<br>
If VPC-2 is happy with the peering connection, then an acknowledgement and acceptance of that request is sent back to the VPC-1.<br>
Both VPCs need to update their routing tables to allow traffic from VPC-1 to get to the destination of VPC-2.

The final part of configuration would be to update security groups that are hosting any resources within the VPCs to be shared.<br>
They need to update the rules to allow the correct resources, ports, and protocols to communication with each other.

# Transit Gateway
Connecting multiple VPCs to data centers and offices that are connected via Direct Connect and VPN requires a lot of management of connections and a lot of gateways.<br>
AWS Transit Gateway allows the connection of this infrastructure -- all the VPCs, all the remote locations over Direct Connect or VPN -- via a central hub.

This allows all the VPCs to communicate with each other and with all the remote locations.

Any additional VPCs or remote locations can be added to the AWS Transit Gateway to be included to connect the rest of the infrastructure.

# Amazon Route 53
## What is Domain Name System?
Domain Name Service (DNS) is a hierarachical distributed naming system for computers, services, or any resources connected to the internet or a private network.

A common analogy used to explain DNS, is that it the address book of the internet, as it can look up a human-friendly name and it will provide the respective IP address.

## Amazon Route 53
Amazon Route 53 is the DNS service offered by Amazon.

* Highly available and scalable domain name system 
* Provides secure and reliable routing of requests, both for services within AWS and infrastructure that is outside of AWS.
* Provides this service through its global network of authoritative DNS servers that reduce latency and can be managed via the management console or API.

## Hosted Zones
A hosted zone is a container that hold information about how you want to route traffic for a domain such as CloudAcademy.com

Route 53 supports the following types of zones:
* Public Hosted Zone - This zone determines how traffic is routed on the internet and can be created when you register your domain with Route 53
* Private Hosted Zone - For Amazon VPC, this zone determines how traffic is routed within the Amazon VPC. If your resources are not accessible outside fo the VPC you can use any domain name you wish.

## Domain Types
There are different domains that are supported by Route 53:
* Generic Top-Level Domains (TLDs) - TLDs are used to help determing what information you might expect to find on the website.
    * **.watch** - may be used for websited relating to streaming, videos, or write watches
    * **.clothing** - may be used by those in the fashion industry as well as designers
* Geographical Domains - These are used to represent the geographical location of the site
    * **.com.au** - Australia
    * **.uk** - UK

## Routing Policies
When you create a resource record set you must choose a routing policy that will be applied to it, and this determines how Route 53 will respond to these queries.

The routing policies available include:
* Simple - Default policy used for single reources that perform a given function.
* Failover - Allows routing to different resources based on their health.
* Geo-location - Traffic is routed based on the geographic location of users.
* Geoproximity - Traffic is routed based on the geographic location of users AND resources.
* Latency - Resources are in multiple regions and low latency is required.
* Multivalue Answer - Gets a response from a DNS request from up to 8 records at once, picked at random.
* Weighted - When multiple resource recoards perform the same function
    * To determing the probability, the formula is the weight of the individual resource record divided by the sum of the total value in resource record set.

# Amazon Cloudfront
Amazon CloudFront is AWS's fault-tolerant and glocally scalable content delivery network service.<br>
It provides a seamless integration with other Amazon Web Service services to provide an easy way to distribute content.

Speeds up distribution of your static and dynamic content through its worldwide network of edge locations.

When using CloudFront, content requests are **routed to the closest edge** to the user's location which provides the lowest latency to deliver the best performance though cached data.

Amazon CloudFront acts as a **content delivery network service**, which provides a means of distributing the source data of web traffic closer to the end-user requesting the content via AWS edge locations as casched data.<br>
Because this is data is cached, it will expire after a set period therefore AWS CloudFront does not provide durability of the data.<br>
Instead it distributes the source data which resides on durable storage, such as S3.

## AWS Edge Locations
AWS edge locations are sites deployed in major cities and highly populated areas across the globe.

You may have your website hosted on EC2 instances or S3 within the Ohio region with an associated CloudFront distribution

When a user accesses the website from Europe, they would be redirected to their closest edge location in Europe, where cached data could be read on the website.

## Distribution
CloudFront uses distributions to control which source data it needs to redistribute and to where.
* Web distribution
    * Speeds up distribution of static and dynamic content
    * Distribute media files using HTTP or HTTPS
    * Add, update, or delete objects, and submit data from web forms.
    * Use live streaming to stream an event in real-time
* RTMP distribution
    * Distribute media with the Adobe Flash media service RTMP protocol
    * The source data for an RTMP distribution can only exist within an S3 bucket, not an EC2 web server.

When configuring distributions, origin information is required.

If the origin is an S3 bucket, then it can be selected from a drop-down list.

When an S3 is used as a static website, enter the static hosting website endpoint.

When using an S3 bucket as the origin, then for additional security a CloudFron user called Origin Access Identity (OAI) can be created.
&emsp;This ensures that only this OAI can access and serve content from the bucket and therefore preventing anyone circumventing the CloudFron distribution by accessing files directly in the bucket using the object URL.

You will also be required to select a host of different caching behaviour options, defining how you want the data at the edge location to be cached via various methods and policies.

You will define the distribution settings themselves, and this will look at which edge locations you want your data to be distributed to:
* US, Canada, Europe
* US, Canada, Europe, Asia
* All edge locations (best performance)

You can also define if you want the distribution to be associated to a web application firewall access control list for additional security and web application protection.

In addition to using a web application firewall access control list, you can implement additional encryption security by specifying an SSL certificate that must be used with a distribution. 

# AWS Global Accelerator
AWS is a **Global** AWS services and is not tied to a specific region.

AWS Global Accelerator aims to get UDP and TCP traffic from end user clients to the applications faster, quicker, and more reliably, through the use of the AWS global infrastucture and specific endpoints.

Global Accelerator uses two static IP addresses associated with a DNS name, used as a fixed source, to gain access to applications which could be sitting behind a load balancer, such as a network or application load balancer, or directly connected to the EC2 instance or the Elastic IP address.<br>
These IP addresses can be mapped to multiple different endpoints, each operating in a different region if a multi-region application is deployed to enhance performance of routing choices.

Because the routing of requests is based across the AWS Global Infrastructure, Global Accelerator intelligently routes customers requests across the most optimised path using it global reach of edge locations, for the lowest latency and avoids unhealthy resources.<br>
This helps improve regional failover and high availability across your deployment.

## Setting up AWS Global Accelerator
1. Create and name the accelerator
    * Assign 2 IP addresses either from AWS' pool or your own
    * Each accelerator requires 2 IP addresses
2. Create a listener
    * Listeners are used to receive and process incoming connections based upon both the protocol and ports specified, which can either be UDP or TCP.
3. Associate the listener with an Endpoint Group.
    * An endpoint group is associated with a different region, and within each group there are multiple endpoints.
        * Optionally set a traffic dial - the desired percentage of traffic to go to the end point. This helps with blue/green deployments of the application to control the amount of traffic to specific regions.
        * Configure health checks to allow the global accelerator to understand what is deemed healthy or unhealthy.
4. Associate and register endpoints for the application. This can either be an application load balancer, a network load balancer, an EC2 instance, or an Elastic IP address.
    * For each endpoint you can also assign a weight to route the percentage of traffic to that endpoint in each of your endpoint groups.